# Fine-tuning data prep

Explore some open-source dataset for fine-tuning a small 3B / 7B model.

Topics involved:
- Dataset download and loading
- Preprocessing : prepare input and output based on FT objective
- Tokenization : tokenize input and output
- Batching : prepare batches for training
- Train / Test split

## 1. Data Prep

For most finetuning tasks, the dataset consists of `input` -> `output` pairs, where the `input` is the prompt and the `output` is the expected response. 
An example of this kind of dataset is the [openassistant-guanaco](https://huggingface.co/datasets/timdettmers/openassistant-guanaco?row=0) dataset where `input` is the human prompt and `output` is the assistant response.

For preference tuning using DPO or RLHF tasks the dataset consists of a comparison pairs of `prompt`, `accepted_target` and `rejected_target` pairs. RLHF datasets only contain `accepted_target` and `rejected_target` pairs with the same prompt.

References 
- [Transformers - Data Preparation for LLM finetuning](https://huggingface.co/docs/autotrain/en/llm_finetuning)
- [Fine-tuning LLMs in 2024](https://www.philschmid.de/fine-tune-llms-in-2024-with-trl)


Goal: Download a few sample datasets and print them out.


1. [OpenAssistant](https://huggingface.co/datasets/timdettmers/openassistant-guanaco?row=0)

* columns: [`text`]
* description: `text` columns is human prompt followed by assistant response, seperated by `###`. Multilingual.

In [1]:
from datasets import load_dataset

In [3]:
oa_dataset = load_dataset("timdettmers/openassistant-guanaco", split='train')
oa_dataset

Repo card metadata block was not found. Setting CardData to empty.


Dataset({
    features: ['text'],
    num_rows: 9846
})

In [5]:
oa_dataset[0]['text']

'### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, lead

2. [DPO dataset](https://huggingface.co/datasets/argilla/dpo-mix-7k)

* columns: `chosen` and `rejected` are preferences for the chosen and rejected responses, respectively based on same prompt. Also contains associated preference scores, which is not directly required.
* description: mix of DPO datasests.

In [7]:
dpo_dataset = load_dataset("argilla/dpo-mix-7k", split='train')
dpo_dataset

Dataset({
    features: ['dataset', 'chosen', 'rejected', 'chosen_rating', 'rejected_rating'],
    num_rows: 6750
})

In [8]:
dpo_dataset[0]

{'dataset': 'argilla/distilabel-intel-orca-dpo-pairs',
 'chosen': [{'content': 'Q:Question: how old julio cesar chavez when he fought de la hoya I found the following answer on Google: He holds records for most successful consecutive defenses of world titles (27), most title fights (37), most title-fight victories (31) and he is after Joe Louis with (23) for most title defenses won by knockout (21). Is that a correct answer? Yes or no.\nA:',
   'role': 'user'},
  {'content': " Yes, the information you found on Google is correct. Julio César Chávez holds several records related to world title defenses and victories, and he is considered one of the greatest boxers in history. Here is a detailed answer to your question:\n\nJulio César Chávez was born on July 12, 1962, in Ciudad Obregón, Sonora, Mexico. He began boxing at a young age and quickly made a name for himself in the sport, winning his first world title in 1984 when he defeated Mario Miranda for the WBC super featherweight champio

## 2. Data Prep

Data Prep is converting the dataset in the format required by the training software.

* Trainig softwares like `tlr` require data in a [particular format](https://www.philschmid.de/fine-tune-llms-in-2024-with-trl).
* If training directly with pytorch as a `CausalLM` task, the data should be formatted as a continous sequence of text. If using a instruct-LLM like mistral-chat, the sequence should be formatted in same format as instruction tuning procedure for optimal performance.

## 3. Tokenization

Use the `tokenizer` associated with the LLM to the trained. 